## Process earnings reports data from saved pickle

E. Quinn 12/20/2020

## Import standard python datascience packages

In [ ]:
import sys
import math
import re
import copy
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

In [ ]:
from datetime import datetime, timedelta, date
from datascience import *
import uuid
import random

### Import custom SC classes

In [ ]:
sys.path.append("/home/gquinn/EG/school_committee/py_EGSC")

In [ ]:
from cba_salaries import Teacher_salaries 
from payperiod import Payperiod
from ucoa_labels import UCOA_labels
from eg_acct_codes import EG_acct_codes
from pay_check import Pay_check, Check_lineitem
from person import Person
from scenario import Scenario

In [ ]:
ts = Teacher_salaries()
help(ts)

### Show the directory we are running in

In [ ]:
!pwd

### Load RIDE UCOA labels 

In [ ]:
UCOA_labels = UCOA_labels()
    
#help(UCOA_labels)

### EG accounting codes class

provides descriptions for EG accounting codes and mapping to UCOA codes

In [ ]:
EG_acct_codes = EG_acct_codes()
    
#help(EG_acct_codes)

### Load teacher salary matrix code

In [ ]:
teacher_salary_matrix = Teacher_salaries()
    
#help(teacher_salary_matrix)

### Load serialized people object produced from earnings report

In [ ]:
#Load data (deserialize)
with open('../../finance_subcommittee/people_12_21_2020.pkl', 'rb') as handle:
    people = pickle.load(handle)
    
len(people)

In [ ]:
people

In [ ]:
file1 = open('../earnings_detail.csv','w')

hstr = '"Name","School Year","Check Date","Check Number","Position","Fund",' \
    + '"Acct","Acct Description","Obj","Obj Description","Earnings",' \
    + '"Step","Payments","FTE","Salary"\n'
file1.write(hstr)

for name in people.keys():
    hstr = '"' + name + '","'
    pp = people[name].get_payperiods()
    for syr in pp.keys():
        hstr2 = hstr + syr + '",'
        for pseq in pp[syr].keys():
            #hstr3 = hstr2 + str(pseq) + ','
            hstr3 = hstr2
            cks = pp[syr][pseq].get_checks()
            for ckey in cks.keys():
                cdate = cks[ckey].get_date()
                cnumbr= cks[ckey].get_number()
                hstr4 = hstr3 + str(cdate) + ',' + str(cnumbr) + ','
                itms = cks[ckey].get_items()
                for itmkey in itms.keys():
                    fund = itms[itmkey].get_fund()
                    acct = itms[itmkey].get_acct()
                    obj  = itms[itmkey].get_obj()
                    pos  = itms[itmkey].get_position()
                    rate = itms[itmkey].get_rate()
                    accdesc = itms[itmkey].get_acct_desc()
                    objdesc = itms[itmkey].get_obj_desc()
                    earn    = itms[itmkey].get_earnings()
                    stepinfo = itms[itmkey].get_stepinfo()
                    if 'step' in stepinfo.keys():
                        step = stepinfo['step']
                    else:
                        step = ''
                    if 'payments' in stepinfo.keys():
                        pmts = stepinfo['payments']
                    else:
                        pmts = ''
                    if 'fte' in stepinfo.keys():
                        fte = stepinfo['fte']
                    else:
                        fte = ''
                    if 'salary' in stepinfo.keys():
                        sal = stepinfo['salary']
                    else:
                        sal = ''
                    hstr5 = hstr4 + pos + ',' + str(fund) + ',' + str(acct) + ',' + \
                    str(accdesc) + ',' + str(obj) + ',' + str(objdesc) + ',' + \
                    str(earn) + ',' + step + ',' + str(pmts) + ',' + str(fte) + ',' + \
                    str(sal) + '\n'
                    file1.write(hstr5)
file1.close()